In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
random_seed = 77

In [ ]:
TRAIN_CSV = '/kaggle/input/new-york-city-taxi-fare-prediction/train.csv'
TEST_CSV = '/kaggle/input/new-york-city-taxi-fare-prediction/test.csv'

train_df =  pd.read_csv(TRAIN_CSV, nrows = 500_000, parse_dates=["pickup_datetime"])

In [ ]:
test_df = pd.read_csv(TEST_CSV)

In [ ]:
train_df.head()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.columns

In [ ]:
train_df.shape

In [ ]:
null_columns = ['dropoff_longitude', 'dropoff_latitude']

In [ ]:
train_num = train_df.shape[0]
test_num = test_df.shape[0]

In [ ]:
null_index = train_df.isnull().any(axis=1)

In [ ]:
null_index.shape

In [ ]:
null_index = null_index[null_index == True]

In [ ]:
null_index

In [ ]:
train_df = train_df.dropna(how = 'any', axis = 'rows')

In [ ]:
train_num = train_df.shape[0]
test_num = test_df.shape[0]

In [ ]:
train_df

In [ ]:
train_df['fare_amount']

In [ ]:
train_df = train_df.assign(longitude_offset =np.abs(train_df['pickup_longitude']-train_df['dropoff_longitude']) )

In [ ]:
test_df = test_df.assign(longitude_offset =np.abs(test_df['pickup_longitude']-test_df['dropoff_longitude']) )

In [ ]:
train_df = train_df.assign(latitude_offset =np.abs(train_df['pickup_latitude']-train_df['dropoff_latitude']) )

In [ ]:
test_df = test_df.assign(latitude_offset =np.abs(test_df['pickup_latitude']-test_df['dropoff_latitude']) )

In [ ]:
train_df.head()

In [ ]:
drop_columns = ['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']

In [ ]:
train_df = train_df.drop(drop_columns,axis = 1)

In [ ]:
test_df = test_df.drop(drop_columns,axis = 1)

In [ ]:
a=2

In [ ]:
from sklearn.model_selection import train_test_split # データセット分割用

In [ ]:
train = train_df.drop('fare_amount',axis = 1)

In [ ]:
train = train.drop('key',axis = 1)
test = test_df.drop('key',axis = 1)

train = train.drop('pickup_datetime',axis = 1)
test = test.drop('pickup_datetime',axis = 1)

In [ ]:
train

In [ ]:
train.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(train,train_df['fare_amount'],test_size = .2,random_state = 2)

In [ ]:
X_test

In [ ]:
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import sem

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train,y_train)
lgb_eval = lgb.Dataset(X_test,y_test,reference = lgb_train)

In [ ]:
'''params = {
    "objective": "regression",
    'metric': {'rmse'},
}
models = []
row_no_list = list(range(len(y_train)))

K_fold = KFold(n_splits = 5,shuffle = True,random_state = random_seed)

for train_cv_no,eval_cv_no in K_fold.split(row_no_list,y_train):
    X_train_cv = X_train.iloc[train_cv_no,:]
    y_train_cv = pd.Series(y_train).iloc[train_cv_no]
    X_eval_cv = X_train.iloc[eval_cv_no,:]
    y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]
    
    lgb_train = lgb.Dataset(X_train_cv,y_train_cv)
    lgb_eval = lgb.Dataset(X_eval_cv,y_eval_cv,reference = lgb_train)
    
    evaluation_results = {}
    model = lgb.train(params,lgb_train,num_boost_round = 1000,valid_names = ['train','valid'],valid_sets = [lgb_train,lgb_eval],evals_result = evaluation_results,early_stopping_rounds = 20,verbose_eval = -1)
    y_pred = model.predict(X_test,num_iteration = model.best_iteration)
    
    #y_pred_max = np.argmax(y_pred,axis = 1)
    print('\n\n\n\n')
    models.append(model) 

'''


In [ ]:
#model = lgb.train(params,train_set=lgb_train,valid_sets = lgb_eval)

In [ ]:
y_train

In [ ]:
import time

In [ ]:
'''
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
start = time.time()

cv_params = {'reg_alpha': [0.0001, 0.1],
             'reg_lambda': [0.0001, 0.1],
             'num_leaves': [4, 6],
             'colsample_bytree': [0.7],
             'subsample': [0.6],
             'subsample_freq': [15],
             'min_child_samples': [5, 10]
             }


clf = LGBMRegressor()
gsearch = GridSearchCV(estimator=clf, param_grid=cv_params)
gsearch.fit(X_train, y_train)
# 最適パラメータの表示と保持
best_params = gsearch.best_params_
best_score = gsearch.best_score_
print(f'最適パラメータ {best_params}\nスコア {best_score}')
print(f'所要時間{time.time() - start}秒')
'''

In [ ]:

from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
best_params = {'boosting_type':'gbdt','colsample_bytree': 0.7, 'min_child_samples': 10, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.0001, 'subsample': 0.6, 'subsample_freq': 15}
clf = LGBMRegressor(**best_params)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
eval_test = clf.predict(X_test)
sqrt(mean_squared_error(eval_test,y_test))

In [ ]:

eval_train = clf.predict(X_train)
sqrt(mean_squared_error(eval_train,y_train))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
plt.scatter(X_test['latitude_offset'],y_test)

In [ ]:
X_test.describe()

In [ ]:
lgb_train

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
y_pred = clf.predict(test)

In [ ]:
train.head()

In [ ]:
a=2

In [ ]:
submission = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv")
submission.shape

In [ ]:
submission.iloc[:,1] = np.floor(y_pred)

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index = False)